In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
import keras.layers as layers
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import pickle

from db_actions import DatabaseActions as db
import tokenizing
from frequency_analysis import get_prob
from structure.train_data import Record, StoredData


print('TensorFlow version: {}'.format(tf.__version__))

2023-06-05 11:29:40.011676: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.12.0


In [2]:
data = db.get_parsed()

storedData = StoredData(data)

fake_len_all:  222
fake_len_train:  155
fake_len_test:  67
not_fake_len_all:  1751
not_fake_len_train:  533
not_fake_len_test:  67


In [3]:
print(storedData.x_train.shape)
print(type(storedData.x_train[0]), len(storedData.x_train[0]))
print(type(storedData.x_train[0][0]))
print(type(storedData.y_train), len(storedData.y_train))
print(type(storedData.y_train[0]))

(688, 768)
<class 'numpy.ndarray'> 768
<class 'numpy.float32'>
<class 'numpy.ndarray'> 688
<class 'numpy.int64'>


In [4]:

# model = Sequential()
# model.add(Dense(768 * 16, activation='elu', input_shape=(768,)))
# model.add(Dense(1024 * 8, activation='elu'))
# model.add(Dense(1024 * 4, activation='elu'))
# model.add(Dropout(0.1))
# model.add(Dense(1024 * 4, activation='elu'))
# model.add(Dense(1024, activation='elu'))
# model.add(Dropout(0.2))
# model.add(Dense(512, activation='elu'))
# model.add(Dense(256, activation='elu'))
# model.add(Dense(256, activation='elu'))
# model.add(Dense(64, activation='elu'))
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=[tf.keras.metrics.AUC(curve='ROC')])


# # model.summary()


# # Train the model
# model.fit(storedData.x_train, storedData.y_train, epochs=20, verbose=1)

In [5]:
x_t = storedData.x_train.reshape((len(storedData.x_train), 24, 32))
x_ts = storedData.x_test.reshape((len(storedData.x_test), 24, 32))

In [75]:
import tensorflow as tf

# Create a simple CNN model
def create_cnn_model(input_shape):
    model = tf.keras.Sequential()
    
    # Convolutional layer with 32 filters, 3x3 kernel size, and ReLU activation
    model.add(layers.Conv2D(256, (9, 9), activation='tanh', input_shape=input_shape))
    
    # Max pooling layer with 2x2 pool size
    model.add(layers.MaxPooling2D((3, 3)))
    
    # Convolutional layer with 64 filters, 3x3 kernel size, and ReLU activation
    model.add(layers.Conv2D(512, (4, 4), activation='tanh'))
    
    # Max pooling layer with 2x2 pool size
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Flatten the output from the previous layer
    model.add(layers.Flatten())
    
    # Fully connected layer with 64 units and ReLU activation
    model.add(layers.Dense(256, activation='tanh'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(64, activation='tanh'))
    
    # Output layer with 1 unit and sigmoid activation for binary classification
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

# Create an instance of the model
input_shape = (24, 32, 1) # Example shape, adjust based on your data
model = create_cnn_model(input_shape)

# Compile the model
# model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9),
#               loss='binary_crossentropy',
#               metrics=[tf.keras.metrics.AUC(curve='ROC')])
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=[ tf.keras.metrics.AUC(curve='ROC'),
                        keras.metrics.Precision(name='precision'),
                        keras.metrics.Recall(name='recall'),
                        keras.metrics.BinaryAccuracy(name='accuracy')])
                                            

# Print the model summary
model.summary()


NameError: name 'layes' is not defined

In [69]:
model.fit(x_t, storedData.y_train, epochs=25, verbose=1)

Epoch 1/25
22/22 [==============================] - 3s 114ms/step - loss: 0.5889 - auc_16: 0.5599
Epoch 2/25
22/22 [==============================] - 3s 118ms/step - loss: 0.4684 - auc_16: 0.7568
Epoch 3/25
22/22 [==============================] - 3s 114ms/step - loss: 0.4308 - auc_16: 0.7853
Epoch 4/25
22/22 [==============================] - 2s 106ms/step - loss: 0.4227 - auc_16: 0.7985
Epoch 5/25
22/22 [==============================] - 2s 108ms/step - loss: 0.4099 - auc_16: 0.8071
Epoch 6/25
22/22 [==============================] - 2s 113ms/step - loss: 0.4082 - auc_16: 0.8190
Epoch 7/25
22/22 [==============================] - 3s 113ms/step - loss: 0.4208 - auc_16: 0.7975
Epoch 8/25
22/22 [==============================] - 3s 117ms/step - loss: 0.4042 - auc_16: 0.8166
Epoch 9/25
22/22 [==============================] - 3s 119ms/step - loss: 0.4076 - auc_16: 0.8197
Epoch 10/25
22/22 [==============================] - 3s 121ms/step - loss: 0.4010 - auc_16: 0.8072
Epoch 11/25
22/22 [

In [70]:
# for i in range(95, 105):
#     print(i, y_train[i], texts[i], sep= '\t', end='\t')
#     print()


res = model.predict(x_ts)
dif = np.sum(res) / np.size(res)

print(dif)
print(storedData.y_test)
c = 0
for i in range(len(x_ts)):
    if (res[i] > dif) == storedData.y_test[i]:
        print("YES", res[i] > dif, storedData.y_test[i])
        c += 1
    else:
        print("NO", res[i] > dif, storedData.y_test[i])
print(c, len(storedData.x_test))
print(c / len(storedData.x_test))

5/5 [==============================] - 0s 15ms/step
0.2952615538639809
[0 1 1 0 0 1 0 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 1 1 0 0 1 0 1 0 0 0 1 1 1 1 0
 1 1 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 0 0 1 1 0 0 1 1 0 1 0 0 0 0 1 1 0
 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0
 0 0 1 1 1 0 1 1 1 0 1 0 1 0 0 1 1 1 0 0 1 1 1]
YES [False] 0
YES [ True] 1
NO [False] 1
NO [ True] 0
YES [False] 0
YES [ True] 1
YES [False] 0
NO [False] 1
YES [False] 0
YES [ True] 1
YES [False] 0
NO [ True] 0
YES [False] 0
YES [False] 0
YES [False] 0
YES [ True] 1
YES [ True] 1
YES [ True] 1
YES [ True] 1
NO [ True] 0
YES [False] 0
YES [False] 0
YES [ True] 1
YES [ True] 1
YES [False] 0
YES [False] 0
YES [ True] 1
YES [False] 0
YES [ True] 1
NO [ True] 0
NO [ True] 0
YES [False] 0
YES [ True] 1
YES [ True] 1
NO [False] 1
YES [ True] 1
YES [False] 0
YES [ True] 1
YES [ True] 1
YES [False] 0
YES [False] 0
YES [ True] 1
NO [False] 1
YES [ True] 1
YES [ True] 1
YES [ True] 1
YES [False] 0
YES [ T

In [71]:
for i in range(len(storedData.x_test)):
    print('(', i, ') ', res[i], res[i] > dif, storedData.y_test[i], sep='\t')



(	0	) 	[0.01977045]	[False]	0
(	1	) 	[0.49628803]	[ True]	1
(	2	) 	[0.02641177]	[False]	1
(	3	) 	[0.49573264]	[ True]	0
(	4	) 	[0.04678861]	[False]	0
(	5	) 	[0.49615267]	[ True]	1
(	6	) 	[0.1799083]	[False]	0
(	7	) 	[0.03508859]	[False]	1
(	8	) 	[0.01831624]	[False]	0
(	9	) 	[0.48263407]	[ True]	1
(	10	) 	[0.03025738]	[False]	0
(	11	) 	[0.49625877]	[ True]	0
(	12	) 	[0.07379356]	[False]	0
(	13	) 	[0.18321289]	[False]	0
(	14	) 	[0.18196613]	[False]	0
(	15	) 	[0.4843428]	[ True]	1
(	16	) 	[0.495828]	[ True]	1
(	17	) 	[0.49579364]	[ True]	1
(	18	) 	[0.495663]	[ True]	1
(	19	) 	[0.47133666]	[ True]	0
(	20	) 	[0.18270022]	[False]	0
(	21	) 	[0.18251453]	[False]	0
(	22	) 	[0.49639526]	[ True]	1
(	23	) 	[0.49631646]	[ True]	1
(	24	) 	[0.01628275]	[False]	0
(	25	) 	[0.01643713]	[False]	0
(	26	) 	[0.4814355]	[ True]	1
(	27	) 	[0.06743386]	[False]	0
(	28	) 	[0.479601]	[ True]	1
(	29	) 	[0.4964132]	[ True]	0
(	30	) 	[0.49594477]	[ True]	0
(	31	) 	[0.05703238]	[False]	0
(	32	) 	[0.496242]	[ True]	1

In [72]:
res = model.predict(x_t)
dif2 = np.sum(res) / np.size(res)
print(dif2)
# 0.9999979073660714
# 0.9999981471470424

# 0.7628571428571429
# 0.7628571428571429
c = 0
for i in range(len(x_t)):
    if (res[i] > dif2) == storedData.y_train[i]:
        c += 1
print(c, len(x_t))
print(c / len(x_t))


22/22 [==============================] - 0s 15ms/step
0.23183946831281796
529 688
0.7688953488372093


In [74]:

def check_review(text):
    # vector = ((tokenizing.tokenize_res(text)) / 4 + 1) / 2
    vector = tokenizing.tokenize_res(text) 
    print(vector.shape)
    vector = vector.reshape((24, 32))
    print('~' * 80)
    # print(vector)
    return(model.predict(np.array([vector,])))

txt1 = "Очень пнравился ресторан, кухня Азербайджанская, примлемые цены, все очень вкусное, чай превосходный, также на 2 этаже есть магазин с Азербайджанскими товарами ручной работы, в общем при посещении этого места, есть ощущение, что посетил Азербайджан"
txt2 = "Превосходный ресторан! Друзья пригласили сюда, остался в восторге! Потрясающая кухня, приятная атмосфера и приветливый персонал. Так держать! Кстати, в туалете очень красивый дизайн"
txt3 = "Наверно, станет одним из любимых мест. Вкусно очень! Я кулинар, поэтому оценила и качество продуктов, и тонкости приготовления. Здорово! Вернёмся. Рекомендую.  Звезду сняла за ОЧЕНЬ громкую и безвкусную (или очень на любителя) музыку.  Господа хозяева-менеджеры, вы прекрасно всё сделали, к вам ходят семьи с маленькими детьми, у вас неплохое детское меню. Не портите имидж, используйте приятную нейтральную музыку с нормальными децибеллами, серьёзный клиентский сектор отпугиваете!"
txt4 = "мне очень понравился этот ресторан. очень классное место, приду еще"
txt5 = "хорошо"

r1 = check_review(txt1)
r2 = check_review(txt2)
r3 = check_review(txt3)
r4 = check_review(txt4)
r5 = check_review(txt5)

print(dif)
print("~" * 80)
print(dif <= r1, r1)
print(dif <= r2, r2)
print(dif <= r3, r3)
print(dif <= r4, r4)
print(dif <= r5, r5)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized because the s

(768,)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1/1 [==============================] - 0s 27ms/step


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized because the s

(768,)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1/1 [==============================] - 0s 27ms/step


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized because the s

(768,)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1/1 [==============================] - 0s 27ms/step


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized because the s

(768,)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1/1 [==============================] - 0s 26ms/step


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized because the s

(768,)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1/1 [==============================] - 0s 35ms/step
0.2952615538639809
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[[False]] [[0.19160731]]
[[False]] [[0.18016975]]
[[False]] [[0.22184788]]
[[False]] [[0.15914516]]
[[False]] [[0.19118807]]


In [20]:
tmp = check_review("идеальшейшая корчма -- она очень сильно понравится. для истинных гурманов")

print(dif <= tmp, tmp, dif)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized because the s

(768,)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1/1 [==============================] - 0s 29ms/step
[[ True]] [[0.4132369]] 0.32472456747026585


In [ ]:
# print(texts)


data = []

for i in range(len(texts)):
    data.append(get_prob(texts[i]))

import matplotlib.pyplot as plt

d1 = np.array(data[0:100])
d1.sort()
# d1 = d1[0:90]

d2 = np.array(data[100:200])
d2.sort()

# d2 = d2[0:90]
# Create a line plot
plt.plot(d1)
plt.plot(d2)

# Add labels and title
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Array Data Visualization')

# Display the plot
plt.show()
print("fake", np.median(d1), np.average(d1))
print("normal", np.median(d2), np.average(d2))

NameError: name 'texts' is not defined

In [ ]:
# np.seterr('raise') 
d3 = []
s = 0.0
for i in range(1, len(d1)):
    print(d1[i], d2[i])
    s = s + d1[i] / d2[i]
    d3.append(s)
print(d3)
plt.plot(d3)
# Add labels and title
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Array Data Visualization')

# Display the plot
plt.show()

In [ ]:
# np.seterr('raise') 
d4 = []
d1[:] = d1[::-1]
d2[:] = d2[::-1]
s = 0.0
for i in range(1, len(d1)):
    s = s + d1[i]
    d4.append(s)

plt.plot(d4)

d5 = []
s = 0.0
for i in range(1, len(d1)):

    s = s + d2[i]
    d5.append(s)

plt.plot(d5)
# Add labels and title
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Array Data Visualization')

# Display the plot
plt.show()

In [ ]:
# import os

# model.summary()
# model.save('model-05062023')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 19, 27, 128)       4736      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 9, 128)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 3, 6, 256)         524544    
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 1, 3, 256)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 768)               0         
                                                                 
 dense_2 (Dense)             (None, 64)               

INFO:tensorflow:Assets written to: model-05062023/assets


INFO:tensorflow:Assets written to: model-05062023/assets
